In [0]:

############################ Import des données ########################

####### Pandas et Numpy.###########
import pandas as pd
import math
import numpy as np

########### Fonctions Spark. ##############
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import DataFrame,Window
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import year,round,substring,udf,mean, col,countDistinct,sum,count,month,max,dayofyear,datediff,months_between,abs,min,isnull,avg

################Plots.#################### 
import seaborn as sn
import matplotlib.pyplot as plt

############### Regression logistiqu,multicolinéaire ###########
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

############ Apprentissage statistique (données manquantes, PCA, Diminution de variance)#################
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_validate, cross_val_score
import scipy.stats as stats

############### Recherche au sein d'un string d'un autre string. #################
import re


In [0]:
def analyse_resultat(objglm):
    """
    Méthode pour déterminer si le modèle créé possède des limites avec notamment des Beta trop élevés.
    Attributs:
    ------------
    objglm : objet issu d'un glm et d'un fit sur un jeu de données.

    Returns:
    ------------
    dict. Dictionnaire répertoriant la valeur de la log-vraisemblance ainsi que la valeur du vif pour chaque colonne.
    """

    Cov=objglm.model.exog
    i=0
    VIF=[]
    liste_noms=[]
    dico_analyse={}
    vif_col=[]
    while i<Cov.shape[1]:
        vif_col=variance_inflation_factor(Cov,i)
        VIF.append(vif_col)
        liste_noms.append(objglm.model.exog_names[i])
        i=i+1
    Table=pd.DataFrame()
    Table["nom"]=liste_noms
    Table["VIF"]=VIF
    dico_analyse["VIF_colonnes"]=Table
    """calculer la log vraisemblance, vérifier si na ou non."""
    loglikeli=objglm.llf
    if loglikeli==np.nan:
        reponse=False
    else:
        reponse=True
    dico_analyse["Finie"]=reponse
    return(dico_analyse)

def vars_a_traduire(df):
    liste_vars=[]
    i=0
    for colonne in df.columns:
        type_var=df.dtypes[i]
        if type_var=='bool' or type_var=="object":
            liste_vars.append(colonne)
        i=i+1
    return(liste_vars)

def vars_explicatives(df):
    """
    Permet dans le cas dans le cas de variables catégorielles et de variables numériques de retrouver la formule utilisée dans les régressions logistiques. 
    ----------------
    Attributs:
    -------
    df : Dataframe de la librairie pandas. 
    ----------
    Returns:
    --------
    chaîne : string. Correspond à l'équation des variables exogènes. 
    """

    chaine=""
    i=0
    if isinstance(df,pd.DataFrame)==False:
        df=pd.DataFrame({df.name:df.values})
    for colonne in df.columns:
        type_var=df.dtypes[i]
        if type_var=='bool' or type_var=="object":
            if i==0:
                """si on a une variable qui est catégorielle (une booléenne ou un object), on devra la transformer en facteur avec le C()"""
                chaine="C("+colonne+")"
            else:
                chaine=chaine+"+C("+colonne+")"
        else:
            if i==0:
                chaine=colonne
            else:
                chaine=chaine+"+"+colonne
        i=i+1
    return(chaine)
def description_influence(objlogit):
    """ 
    Méthode permettant de connaître les effets des différentes variables sur le odd-ratio. 
    --------------
    Attributs:
    --------
    objlogit : objet issu d'une régression logistique. 
    -------------
    Returns:
    --------
    dico_description : dict. Les trois clés correspondent aux effets des variables, les noms des variables
    ayant un effet positif ou à l'inverse négatif.
    """

    dico_description={}
    liste_positif=[]
    liste_negatif=[]
    dictionnaire_beta={}
    liste_noms=objlogit.model.exog_names
    i=0
    mot="Début de la description des effets des variables.\n ##################### \n"
    while i<len(liste_noms):
        dictionnaire_beta[str(liste_noms[i])]=objlogit.params[str(liste_noms[i])]
        i=i+1
    for key,value in dictionnaire_beta.items():
        try :
            valeur=math.exp(value)
            mot=mot+"La variable "+key+ " multiplie le odd_ratio par "+str(valeur)+".\n"
            if math.exp(value)<1:
                liste_negatif.append(key)
            else:
                liste_positif.append(key)
        except:
            """dans le cas où le exponentiel est trop grand, on met dans la liste_positif."""
            mot=mot+"La variable "+key+ " multiplie le odd_ratio par exp("+str(value.round(2))+").\n"
            liste_positif.append(key)
        
    mot=mot+"######################\n Fin de la description des effets."
    dico_description["valeurs_beta"]=dictionnaire_beta
    dico_description["description_generale"]=mot
    dico_description["noms_lien_positif"]=liste_positif
    dico_description["noms_lien_negatif"]=liste_negatif
    return(dico_description)

def  bilan(objlogit):
    """
    Faire un bilan du modèle en fournissant les odd-ratios pour chaque ratio mais aussi la significativité des variables. 
    Attributs: 
    ----
    objlogit : objet issu de la méthode d'un modèle glm (logistique).

    Return :
    ---------
    Pandas Dataframe avec le nom de la variable, la valeur du beta, le odd-ratio et la significativité de la variable. 
    """
    
    Bilan=pd.DataFrame()
    Resultat_p=(objlogit.pvalues<0.05)
    Significative=Resultat_p.values.tolist()
    Nom_variable=Resultat_p.index.tolist()
    Table_significative=pd.DataFrame({"Nom_variable":Nom_variable,"Significativité":Significative})
    dico=description_influence(objlogit).get("valeurs_beta")
    Bilan["Nom_variable"]=dico.keys()
    Bilan["Valeur_beta"]=dico.values()
    Bilan["odd_ratio"]=np.exp(Bilan["Valeur_beta"]).round(2)
    Bilan["Valeur_beta"]=Bilan["Valeur_beta"].round(2)
    Bilan["Significativité"]=Table_significative["Significativité"]
    return(Bilan)


def predire_categorie(df,objet_glm):
    """ Prédire la catégorie de la personne pour une nouvelle base de données. 
    -----------------
    Attributs:
    ----------
    df : Dataframe pandas. Correspond à la base de données de test. 
    objet_glm. Objet issu d'un glm.fit (pour l'instant issu de SMF.fit)
    ---------
    Returns:
    -----
    Vecteur des prédictions. 
    """
    try:
        """Il peut arriver que le code ne fonctionne pas (Erreur : DataFrame has not attribute dtype). """
        predictions=objet_glm.predict(sm.add_constant(df))
    except:
        """Si le transform ne fonctionne pas."""
        ensemble_cat=vars_a_traduire(df)
        for colonne in ensemble_cat:
            """Pour les variables catégorielles, on a besoin de retrouver les bonnes noms de colonnes ainsi que les indicatrices."""
            var_factor=pd.get_dummies(Test[colonne])
            for mod in var_factor.columns:
                """on adopte la manière de coder les variables catégorielles de smf"""
                df["C("+str(colonne)+")[T."+str(mod)+"]"]=var_factor[mod]
        df=df.drop(columns=ensemble_cat)
        """on a plus besoin des anciennes colonnes """
        #Objet model venant de l'ancien glm. 
        """Pour les variables catégorielles, on utilise toujours une modalité de référence. On a donc besoin de filtrer les colonnes."""
        colonnes_glm=objet_glm.model.exog_names
        """l'intersection est réalisée grâce à la méthode intersection du set."""
        liste_cols_final=list(set(df.columns).intersection(set(colonnes_glm)))
        predictions=objet_glm.predict(sm.add_constant(df[liste_cols_final]),transform=False)
    return(predictions)

def selection_variables(df,methode,colonnes_x,colonne_y):
    """ Donner les variables permettant de réduire le plus possible le BIC.  
    Package utilisé : statmodels.
    -----------------
    Attributs:
    ----------
    df : Dataframe pandas. Correspond à la base de données d'appentissage.
    methode : string. Correspond à la méthode utilisée pour obtenir le meilleur résultat. 
    colonnes_x: list. Liste des variables x utilisées. 
    colonne_y : nom de la variable à expliquer dans la base de données. 
    ---------
    Returns:
    -----
    Noms des variables à utiliser. 
    """
    if methode=="forward":
        var="1"
        liste_vars_utilisees=[]
        bic=smf.glm(colonne_y+"~"+var,df,family=sm.families.Binomial()).fit().bic_deviance
        arret=False
        while arret==False:
            if len(liste_vars_utilisees)==0:
                nom_variable="1"
                for variable in colonnes_x:
                    var_nouvelle=vars_explicatives(df[[variable]])
                    bic_nouveau=smf.glm(colonne_y+"~"+var_nouvelle,df,family=sm.families.Binomial()).fit().bic_deviance
                    if bic_nouveau<bic:
                        bic=bic_nouveau
                        nom_variable=variable
                if nom_variable=="1":
                    arret=True
                else:
                    liste_vars_utilisees.append(nom_variable)   
            else:
                nom_variable=liste_vars_utilisees[-1]
                vars_exp_bic=vars_explicatives(df[liste_vars_utilisees])
                bic=smf.glm(colonne_y+"~"+vars_exp_bic,df,family=sm.families.Binomial()).fit().bic_deviance
                for variable in colonnes_x:
                    if variable not in liste_vars_utilisees:
                        vars_exp=vars_explicatives(df[liste_vars_utilisees])
                        var_nouvelle=vars_explicatives(df[[variable]])
                        bic_nouveau=smf.glm(colonne_y+"~"+vars_exp+"+"+var_nouvelle,df,family=sm.families.Binomial()).fit().bic_deviance
                        """Au moins une variable permet de réduire le bic donc on conserve son nom."""
                    else:
                        bic_nouveau=smf.glm(colonne_y+"~"+var_nouvelle,df,family=sm.families.Binomial()).fit().bic_deviance
                    if bic_nouveau<bic:
                        bic=bic_nouveau
                        nom_variable=variable
                if nom_variable==liste_vars_utilisees[-1]:
                    arret=True
                else:
                    liste_vars_utilisees.append(nom_variable)
        return(liste_vars_utilisees)
    
    elif methode=="backward":
        liste_vars_enlevees=[]
        var=vars_explicatives(df[colonnes_x])
        bic=smf.glm(colonne_y+"~"+var,df,family=sm.families.Binomial()).fit().bic_deviance
        arret=False
        while arret==False:
            j=0
            vars_exp_bic=vars_explicatives(df[colonnes_x].drop(columns=liste_vars_enlevees))
            bic=smf.glm(colonne_y+"~"+vars_exp_bic,df,family=sm.families.Binomial()).fit().bic_deviance
            for variable in colonnes_x:
                if variable not in liste_vars_enlevees:
                    vars_exp=vars_explicatives(df[colonnes_x].drop(columns=liste_vars_enlevees).drop(columns=variable))
                    bic_nouveau=smf.glm(colonne_y+"~"+vars_exp,df,family=sm.families.Binomial()).fit().bic_deviance
                    if bic_nouveau<bic:
                        """ au moins une variable va pouvoir être enlevée."""
                        j=1
                        bic=bic_nouveau
                        nom_variable=variable
            if j==0:
                arret=True
            else:
                liste_vars_enlevees.append(nom_variable)
        return(list(set(colonnes_x).symmetric_difference(set(liste_vars_enlevees))))

def selection_variables_cv(df,methode,model, colonnes_x,colonne_y,K,NB,graine,seuil):
    """
    Sélectionner les meilleures variables en n'utilisant plus le critère du BIC mais plutôt les performances au sein de la validation croisée. L'échantillon est celui d'apprentissage.
    Il ne doit pas contenir l'échantillon de test. 
    Package utilisé : sklearn. 

    Attributs :
    ------------
    df : Pandas Dataframe. Echantillon d'appentissage.
    methode : str. Méthode de la sélection de variables.
    model : Méthode. Modèle utilisé.
    colonnes_x : list de str. Variables explicatives.
    colonne_y : str, nom de la variable à expliquer.
    K : int. Nombre de blocs considérés à chaque tour.
    NB : int. Nombre de répétitions. 
    graine : int. Graine aléatoire.
    seuil : float. L'ajout d'une variable doit au moins augmenter le score d'au moins le seuil.

    Return :
    --------
    list. Liste des colonnes contenant les variables sélectionnées.
    """
    if methode=="forward": 
       liste_vars_utilisees=[]
       arret=False
       while arret==False:
            """On s'arrêtera si on ne parvient pas à augmenter le score avec l'ajout d'une nouvelle variable, d'où l'égalité entre nom_variables et nom_derniere_variable."""
            if len(liste_vars_utilisees)==0:
                nom_derniere_variable=" "
                Score_moyen=0
            else:
                nom_derniere_variable=liste_vars_utilisees[-1]
                """ Variables explicatives, sélectionnées dans les tours précédents."""
                liste_variables=list(liste_vars_utilisees)
                """ Variable à expliquer."""
                liste_variables.append(colonne_y)
                print(liste_variables)
                Score_moyen=calcul_score(df[liste_variables],model,K,NB,graine,colonne_y).mean()
            nom_variable=nom_derniere_variable
            for variable in colonnes_x:
                modele=model
                print(variable)
                if variable not in liste_vars_utilisees:
                    if len(liste_vars_utilisees)==0:
                        table=pd.DataFrame({colonne_y:df[colonne_y].tolist(),variable:df[variable].tolist()})
                        Liste_score=calcul_score(table,modele,K,NB,graine,colonne_y)
                    else:
                        liste=list(liste_vars_utilisees)
                        liste.extend([variable,colonne_y])
                        print("Les variables utilisées sont pour rappel "+str(liste))
                        table=pd.DataFrame(df[liste])
                        Liste_score=calcul_score(table,modele,K,NB,graine,colonne_y)
                    Nouveau_score=Liste_score.mean()
                    if(Nouveau_score>=Score_moyen+seuil):
                        """Nom_derniere_variable ne vaut plus nom_variable si on a une variable donne un meilleur score moyen."""
                        nom_variable=variable
                        Score_moyen=Nouveau_score
                        print(str(Score_moyen)+ " comme score avec l'ajout de "+nom_variable)
            if nom_derniere_variable==nom_variable:
                arret=True
            else:
                print(nom_variable)
                liste_vars_utilisees.append(nom_variable)
       return(liste_vars_utilisees)
    
    elif methode=="backward":
        liste_vars_enlevees=[]
        modele=model
        LISTE=list(colonnes_x)
        LISTE.append(colonne_y)
        Score_moyen=calcul_score(df[LISTE],modele,K,NB,graine,colonne_y).mean()
        arret=False
        while arret==False:
            j=0
            Score_moyen=calcul_score(df[LISTE].drop(columns=liste_vars_enlevees),modele,K,NB,graine,colonne_y).mean()
            for variable in colonnes_x:
                if variable not in liste_vars_enlevees:
                    Nouveau_score=calcul_score(df[LISTE].drop(columns=liste_vars_enlevees).drop(columns=variable),modele,K,NB,graine,colonne_y).mean()
                    if (Nouveau_score>=Score_moyen+seuil):
                        """ au moins une variable va pouvoir être enlevée."""
                        j=1
                        print(Nouveau_score)
                        Score_moyen=Nouveau_score
                        nom_variable=variable
            if j==0:
                arret=True
            else:
                liste_vars_enlevees.append(nom_variable)
        return(list(set(colonnes_x).symmetric_difference(set(liste_vars_enlevees))))

def calcul_score(table,model,K,NB,graine,colonne_y):
    """
    Calculer le score du modèle pour les K blocs.
    
    Attributs :
    -----
    table : pandas Dataframe.
    model : estimateur compatible avec Sklearn.
    K: int. Nombre de blocs créés au sein de l'échantillon d'apprentissage.
    NB : nombre de répétitions
    graine : int. Graine aléatoire.
    colonne_y : str. Nom de variable à expliquer.

    Return: 
    ------
    list de float. Correspond au score pour toutes les boucles.
    """
    Table_explicatives=table.drop(columns=colonne_y)
    cv=RepeatedKFold(n_splits=K,n_repeats=NB,random_state=graine)
    cols_a_traduire=vars_a_traduire(Table_explicatives)
    y=table[colonne_y]
    if len(cols_a_traduire)>0:
        table_pour_CV=Table_explicatives[list(set(Table_explicatives.columns).symmetric_difference(cols_a_traduire))]
        for nom_col in cols_a_traduire:
            colonne=pd.get_dummies(Table_explicatives[nom_col])
            for modalite in colonne.columns:
                table_pour_CV.loc[:,str(nom_col)+str(modalite)]=colonne[modalite]
    else :
        table_pour_CV=Table_explicatives
    score_rate=cross_val_score(model,table_pour_CV,y,scoring='roc_auc',cv=cv)
    return(score_rate)

In [0]:
############## Fonction PCA ############
def fonction_pca(df,n):
  """ 
  Permet de renvoyer la projection sur le plan factoriel des variables. Renvoie dans un dictionnaire la transformation du dataframe et les coordonnées des variables.
  df : Dataframe (pandas). 
  n: int
  ----------------------
  Returns:
  -------
  liste_PCA : dict contenant la transformation du dataframe ainsi que les coordonnées des variables.
  """
  df=pd.DataFrame(preprocessing.scale(df),columns=df.columns)
  pca=PCA(n)
  axes=pca.fit_transform(df)
  for i in range(0,n):
    df.loc[:,'PC'+str(i+1)]=axes[:,i]
  pca.fit(df)
  print(pca.explained_variance_ratio_)
  #Prendre le maximum de chaque ligne avec .max(axis=1).
  resultat=pca.components_.max(axis=1)
  (fig, ax) = plt.subplots(figsize=(8, 8))
  nb_variables=pca.components_.shape[1]
  circle=plt.Circle((0,0),resultat[0]+0.10,facecolor="none",edgecolor='k')
  for i in range(0, pca.components_.shape[1]):
      ax.arrow(0,
              0,  # Start the arrow at the origin
              #projection des variables sur le plan généré par les deux vecteurs propres. 
              pca.components_[0, i],  #0 for PC1
              pca.components_[1, i],  #1 for PC2
              head_width=0.1,
              head_length=0.1)
      ax.arrow(0,0,-resultat[0],0,head_width=0,
              head_length=0)
      ax.arrow(0,0,0,-resultat[1],head_width=0,
              head_length=0)
      ax.arrow(0,0,resultat[0],0,head_width=0,
              head_length=0)
      ax.arrow(0,0,0,resultat[1],head_width=0,
              head_length=0)
    #Mettre les noms des colonnes au bon endroit. 
      plt.text(pca.components_[0, i] + 0.02,
              pca.components_[1, i] + 0.02,
              df.columns.values[i])
      ax.add_patch(circle)
 
  print(pd.DataFrame(pca.components_,columns=df.columns))
  dico={}
  dico["donnees_modifiees"]=axes
  dico["coord"]=pca.components_
  return(dico)
  
######### fonction _acm dans le cas où on a des variables ordinales. #############
# ########################### 
def fonction_ACM(data_base,nb_axes):
    """Permet de renvoyer la projection du dataframe sur les nb_axes, de visualiser la projection sur les deux premiers axes. Cela permet de renvoyer les coordonnées des variables. 
    ---------
    Attributs:
    ----
    data_base : DataFrame (pandas). 
    nb_axes : int. Nombre d'axes factoriels considéré. 
    ---------
    Returns :
    ---- 
    dico_acm : dict. Contient les coordonnées des variables et la projection sur les nb_axes.
    """
   
    Table=pd.DataFrame()
    for nom_col in data_base.columns:
        colonne=pd.get_dummies(data_base[nom_col])
        for modalite in colonne.columns:
            Table.loc[:,nom_col+"-"+str(modalite)]=colonne[modalite]
    data_normalise=pd.DataFrame(preprocessing.scale(Table),columns=Table.columns)
    Acm_declare=PCA(nb_axes)
    #L'ACM peut bloquer si on fractionne la base. 
    axes=Acm_declare.fit_transform(data_normalise)
    #Creation des deux axes avec le fit_transform.
    #On aura autant d'axes que de n_components dans la formule PCA(n_components).
    #Le components donne les coordonnées sur le plan des variables. 
    for i in range(0,nb_axes):
        data_normalise.loc[:,'PC'+str(i+1)]=axes[:,i].copy()
    Acm_declare.fit(data_normalise)
    print(Acm_declare.explained_variance_ratio_)
    #Creation de la figure.
    (figure, ax) = plt.subplots(figsize=(10, 10))
    n=Acm_declare.components_.shape[1]
    #Note : comme chez python pour les listes, pandas commence l'indentation à 0.
    #[indice,:] pour prendre la liste des coordonnées pour la première dimension (: pour prendre toutes les colonnes).
    maximum_x=Acm_declare.components_[0, n-2]
    maximum_y=Acm_declare.components_[1, n-1]
    circle=plt.Circle((0,0),maximum_x-0.05,facecolor="none",edgecolor='k')
    for j in range(0,Acm_declare.components_.shape[1]):
        if j==n-1 or j==n-2:
                ax.arrow(0,0,Acm_declare.components_[0, j], 
                Acm_declare.components_[1, j],head_width=0.1,
                head_length=0.1)
        else:
            ax.plot(
                    #projection des variables sur le plan généré par les deux vecteurs propres. 
                    Acm_declare.components_[0, j],  #0 for PC1
                    Acm_declare.components_[1, j],'ro' #1 for PC2
                    )
            ax.arrow(0,0,-maximum_x,0,head_width=0,
                  head_length=0)
            ax.arrow(0,0,0,-maximum_y,head_width=0,
                  head_length=0)
        #Mettre les noms des colonnes au bon endroit. 
            plt.text(Acm_declare.components_[0, j] + 0.01,
                     Acm_declare.components_[1, j] + 0.01,
                     Table.columns.values[j])
            plt.text(0.5,0.05,"Axe 1 : "+str(Acm_declare.explained_variance_ratio_[0].round(2)))
            plt.text(0.05,0.5,"Axe 2 : "+str(Acm_declare.explained_variance_ratio_[1].round(2)))
            plt.xlim((-maximum_x,maximum_x))
            plt.ylim((-maximum_y,maximum_y))
            ax.add_patch(circle)
    dico_acm={}
    dico_acm["donnees_modifiees"]=axes
    dico_acm["coord"]=Acm_declare.components_
    return(dico_acm)



In [0]:
def transforme_prlda(df,NB,cols_,col_y):
    """
    Permet de convertir pour la prédiction ou la création d'un modèle un dataframe. 
    Attributs:
    -----------------
    df : Dataframe pandas. 
    NB : nombre d'axes de l'ACM. 
    cols_: list. Les noms des variables étudiées. 
    Return:
    -----------------
    Dataframe prêt pour la LDA. 
    """
    df2=df[cols_]
    liste_vars=vars_a_traduire(df2.drop(columns=[col_y]))
    if len(liste_vars)>0:
        Donnees_categorielles=df2[liste_vars]
        Donnees_traduites=fonction_ACM(Donnees_categorielles,nb_axes=NB)["donnees_modifiees"]
        print(type(Donnees_traduites))
        i=0
        Data_pre_LDA=df2[vars_a_conserver(df2)]           
        #### Traduction en numérique des variables cat.#####
        while (i<NB):
            Data_pre_LDA.loc[:,"acm-"+str(i+1)]=Donnees_traduites[:,i].copy()
            i=i+1
        Data_LDA=Data_pre_LDA[vars_a_conserver(Data_pre_LDA)].copy()
        sc=StandardScaler()
        Data_LDA=sc.fit_transform(Data_LDA)
        return(Data_LDA)
    else:
        sc=StandardScaler()
        Data_LDA=sc.fit_transform(df2)
        return(Data_LDA)

def modele_lda(df,NB,cols_,col_y):
    """ [[[[ Ce n'est qu'un essai avec une méthode alternative.]]]] 
    Réaliser une analyse linéaire discriminante. 
    -------
    Attribut:
    ---------
    df : Dataframe pandas. 
    NB : nombre d'axes de l'ACM. 
    cols_: list. Liste des noms des colonnes. 
    col_y. Str. Nom de la variable à prédire. 

    Returns:
    ------
    Renvoie un dictionnaire pour résumer le modèle. 
    """
    Data_LDA=transforme_prlda(df,NB,cols_,col_y)
    x=Data_LDA
    y=df[col_y].to_numpy()
    LDA=LinearDiscriminantAnalysis()
    LDA.fit(x,y)
    dico_resultat_lda={}
    dico_resultat_lda["modele"]=LDA
    dico_resultat_lda["x_apprentissage"]=x
    dico_resultat_lda["y_apprentissage"]=y
    return(dico_resultat_lda)

def projection_lda(df,objet_lda):
    """
    Projeter les individus sur le plan scalaire avec les couleurs associées à Y.
    Attributs:
    -----------
    df : pandas Dataframe. Données présentes (X,Y). Données de test ou d'apprentissage. 
    objet_lda : objet issu de modèle lda.

    Returns:
    -------
    Visuel dans un repère avec les individus.
    """

In [0]:
def traiter_multicolinéarité(objglm,data,colonne_y,colonnes_x,valeur_vif_max=5):
    """
    La méthode permet de retirer les variables qui ont un vif strictement supérieur à un certaine valeur. Notons que la valeur du vif pour la constante n'est pas importante.
    Comme le VIF peut rapidement être diminué, on prend variable par variable. 
    Attributs :
    ----------
    objglm : objet issu d'un smf.glm, d'une régression logistique. Cela renvoie à un smf.glm(...).fit()
    data : DataFrame pandas correspondant à l'échantillon d'apprentissage.
    colonne_y : str. Nom de la variable à prédire.
    colonnes_x : str. Nom des variables explicatives.
    valeur_vif_max : float. Il correspond à la valeur maximale autorisée de VIF.
    
    Return:
    -------
    dictionnaire. Trois valeurs: la valeur du vif actuel, un objet glm avec le problème traité et le nom des colonnes.
    """
    """Liste des colonnes à enlever."""
    dictionnaire_vif={}
    Table_vif=analyse_resultat(objglm).get("VIF_colonnes")
    sous_table=Table_vif[(Table_vif["VIF"]>5)&(Table_vif["nom"]!="Intercept")]
    if sous_table["nom"].count()>0:
        liste_drop=[]
        condition=False
        while condition==False:
            maximum=sous_table["VIF"].max()
            Nom_maximum=sous_table[sous_table["VIF"]==maximum]["nom"].to_list()[0]
            if re.search('C(.+)[T.+]',Nom_maximum):
                """Si on a une variable catégorielle, nous devons retrouver dans le data la bonne colonne."""
                for colonne in data.columns:
                    if colonne in Nom_maximum:
                        liste_drop.append(colonne)
                        break
            else:
                liste_drop.append(Nom_maximum)
            Nouvelle_liste_variable=list(set(colonnes_x).symmetric_difference(set(liste_drop)))
            formule_x=vars_explicatives(data[Nouvelle_liste_variable])
            nouvglm=smf.glm(colonne_y+"~"+formule_x,data,family=sm.families.Binomial()).fit()
            nouv_table=analyse_resultat(nouvglm).get("VIF_colonnes")
            sous_table=nouv_table[(nouv_table["VIF"]>5)&(nouv_table["nom"]!="Intercept")]
            if sous_table["nom"].count()==0:
                condition=True
        dictionnaire_vif["GLM"]=nouvglm
        dictionnaire_vif["resultat_vif"]=nouv_table
        dictionnaire_vif["nom_colonnes"]=list(set(colonnes_x).intersection(set(Nouvelle_liste_variable)))
    else:
        print("Aucun changement à réaliser.")
        dictionnaire_vif["GLM"]=objglm
        dictionnaire_vif["resultat_vif"]=Table_vif
        dictionnaire_vif["nom_colonnes"]=colonnes_x
    return(dictionnaire_vif)


def courbe_ROC(nom_col_pred,nom_col_realite,table,valeur,couleur="blue"):
    """
    Construire la courbe ROC. 
    -------------
    Attributs:
    -------
    nom_col_pred : string. nom dans la table "table" de la prédiction. 
    nom_col_realite : string. Nom dans la table "table" de la réalité (vrai y).
    table : DataFrame. Table de test du modèle. 
    valeur : int. Valeur pour laquelle Y est dit positive. 
    couleur : str. Couleur de la droite.
    Return:
    --------
    Graphique de la courbe ROC
    """
    fpr, tpr, thresholds = metrics.roc_curve(table[nom_col_realite], table[nom_col_pred], pos_label=valeur)
    """ fpr "Increasing false positive rates". Proportion de y_chapeau=1 pour les individus n'utilisant pas les téléservices. """
    """  tpr "Increasing true positive rates". Proportion de y_chapeau=1 pour les individus utilisant les téléservices. """
    """ Dans notre cas, le tpr sera parfois un vecteur de 1."""
    (figure, ax) = plt.subplots(figsize=(7, 7))
    point0=np.asarray([0,0])
    point1=np.asarray([1,1]) 
    plt.plot(fpr,tpr,couleur)
    #Droite du hasard.
    plt.axline((0,0),(1,1))
    plt.xlim(0,1)
    plt.ylim(0,1)
def liste_courbe_ROC(liste_cols_pred,nom_col_realite,table,valeur,liste_couleurs):
    """
    Représenter plusieurs courbes ROC dans le même repère. 
    Attributs:
    ----------
    liste_cols_pred: list d'éléments string.  Chaque chaîne de caractères est une colonne issue de la prédiction d'un modèle. 
    nom_colm_realite : string. Nom de la colonne de la variable expliquée dans la table. 
    table : Dataframe pandas. table de test fournissant le X et le Y. 
    liste_couelurs : list de string. Correspond à une liste de couleurs utilisée pour distinguer chaque modèle. 

    Return:
    ---------
    Graphique récapitulant la performance de chaque modèle en terme de sensibilité, spécificité. 
    """
    i=0
    (figure, ax) = plt.subplots(figsize=(7, 7))
    for prediction in liste_cols_pred:
        couleur=liste_couleurs[i]
        fpr, tpr, thresholds = metrics.roc_curve(table[nom_col_realite], table[prediction], pos_label=valeur)
        plt.plot(fpr,tpr,couleur,label=prediction)
        plt.legend()
        plt.title("Courbe ROC sur le nouvel échantillon")
        i=i+1
    plt.axline((0,0),(1,1),label="hasard")
    plt.legend()
    plt.xlim(0,1)
    plt.ylim(0,1)

In [0]:
def convertir_pourSklearn(df,drop_one_mod=False):
    """ Convertir une table dataframe en une table pour sklearn. L'option d'enlever une modalité est disponible. Cela est nécessaire pour retrouver des résultats proches de ceux 
    de statmodels.
    Paramètres :
    -----------
    df : Pandas dataframe. 
    drop_one_mod : Bool. Indique si on enlève une modalité. 
    """

    Table_explicatives=df
    cols_a_traduire=vars_a_traduire(Table_explicatives)
    if len(cols_a_traduire)>0:
        table_sk=Table_explicatives[list(set(Table_explicatives.columns).symmetric_difference(cols_a_traduire))]
        for nom_col in cols_a_traduire:
            colonne=pd.get_dummies(Table_explicatives[nom_col],drop_first=drop_one_mod)
            for modalite in colonne.columns:
                table_sk.loc[:,str(nom_col)+str(modalite)]=colonne[modalite]
    else :
        table_sk=Table_explicatives
    return(table_sk)
